In [1]:
%%writefile lenghtcounts.py

from mrjob.job import MRJob
from mrjob.protocol import RawValueProtocol
from mrjob.step import MRStep
from collections import defaultdict
from json import dumps
import os

class LenCounts(MRJob):
    OUTPUT_PROTOCOL = RawValueProtocol
    
    def steps(self):
        return [
            MRStep(mapper=self.mapper,
                   combiner=self.combiner,
                   reducer=self.reducer),
            MRStep(reducer=self.reducer2)
        ]
    
    def mapper(self, _, line):
        file_name = os.getenv('mapreduce_map_input_file')
        words=line.split()
        for word in words:
            yield((file_name, len(word)), 1)
                    
    def combiner(self, fn_len, cnt_itr):
        yield(fn_len, sum(cnt_itr))
    
    def reducer(self, fn_len, cnt_itr):
        yield(fn_len[0], (fn_len[1], sum(cnt_itr)))
        
    def reducer2(self, fn, len_cnt_itr):
        json_dict={}
        json_dict["file"]=fn.split('//')[1]

        for data in len_cnt_itr:
            ln=data[0]
            cnt=data[1]
            json_dict[str(ln)]=cnt
            
        yield(None, dumps(json_dict))
        
        
        
if __name__ == '__main__':
    LenCounts.run() 

Overwriting lenghtcounts.py


In [2]:
%run lenghtcounts.py americana.txt* 

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/lenghtcounts.tuoms.20191127.180020.981889
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/lenghtcounts.tuoms.20191127.180020.981889/output
Streaming final output from /tmp/lenghtcounts.tuoms.20191127.180020.981889/output...


{"file": "americana.txt5", "24": 1, "29": 1, "2": 17555, "3": 20217, "4": 13898, "5": 11287, "6": 9209, "7": 8348, "8": 5855, "9": 4259, "10": 2644, "11": 1550, "12": 870, "13": 433, "14": 168, "15": 61, "16": 21, "17": 13, "18": 1, "19": 3, "1": 3604, "21": 2}
{"file": "americana.txt7", "10": 2943, "11": 1505, "12": 971, "13": 419, "5": 10729, "6": 8734, "7": 7835, "8": 6013, "9": 4355, "14": 203, "15": 87, "16": 30, "17": 15, "18": 9, "19": 3, "1": 3207, "20": 1, "24": 1, "27": 1, "2": 18593, "3": 20054, "4": 14292}
{"file": "americana.txt4", "19": 3, "1": 3142, "21": 2, "23": 2, "2": 17667, "3": 20870, "4": 13506, "5": 11196, "6": 8802, "7": 8341, "8": 6130, "9": 4372, "10": 2806, "11": 1589, "12": 912, "13": 383, "14": 168, "15": 78, "16": 18, "17": 1, "18": 12}
{"file": "americana.txt2", "4": 13448, "5": 10332, "6": 9263, "7": 7924, "8": 6196, "9": 4810, "10": 3181, "11": 1788, "12": 971, "13": 459, "14": 218, "15": 65, "16": 19, "17": 11, "18": 2, "19": 4, "1": 2924, "20": 2, "21

Removing temp directory /tmp/lenghtcounts.tuoms.20191127.180020.981889...
